## 0. Import Libraries and Set Up

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')

## 1. Load Data

Find and load the necessary datasets for model development.

In [ ]:
# Load owned games data (with genres from Store API)
try:
    owned_games = pd.read_csv('../data/owned_games_enriched.csv')
    print(f"✅ Loaded {len(owned_games)} owned games")
    print(f"Columns: {list(owned_games.columns)}")
    display(owned_games.head(3))
except FileNotFoundError:
    print("❌ Error: Run src/data_collection.py first to collect your Steam data")
    owned_games = pd.DataFrame()

# Load Steam catalog (with genres AND tags from SteamSpy)
try:
    steam_catalog = pd.read_csv('../data/steam_catalog_detailed.csv')
    print(f"\n✅ Loaded {len(steam_catalog)} games from Steam catalog")
    print(f"Columns: {list(steam_catalog.columns)}")
    display(steam_catalog.head(3))
except FileNotFoundError:
    print("❌ Error: Run src/get_detailed_catalog.py to get Steam catalog")
    steam_catalog = pd.DataFrame()

## 2. Data Exploration

## 2.1 Parse Genre and Tag Data

Genres and tags are stored as strings/JSON - we need to parse them for analysis

In [ ]:
import json
import ast

def parse_genres(genres_str):
    """Parse genres from owned_games (list format like ['Action', 'RPG'])"""
    if pd.isna(genres_str) or genres_str == '[]':
        return []
    try:
        # Try parsing as list
        return ast.literal_eval(genres_str) if isinstance(genres_str, str) else genres_str
    except:
        # Fallback: split by comma
        return [g.strip() for g in str(genres_str).split(',') if g.strip()]

def parse_tags(tags_str):
    """Parse tags from catalog (JSON dict like {"FPS": 1234, "Shooter": 567})"""
    if pd.isna(tags_str) or tags_str == '{}':
        return {}
    try:
        return json.loads(tags_str) if isinstance(tags_str, str) else tags_str
    except:
        return {}

def parse_catalog_genre(genre_str):
    """Parse genre from catalog (comma-separated like 'Action, RPG')"""
    if pd.isna(genre_str) or genre_str == '':
        return []
    return [g.strip() for g in str(genre_str).split(',') if g.strip()]

# Parse owned games genres (now uses 'genre' from catalog, not 'genres' from Steam API)
if not owned_games.empty and 'genre' in owned_games.columns:
    owned_games['genres_list'] = owned_games['genre'].apply(parse_catalog_genre)
    print("✅ Parsed owned games genres (from catalog)")
    print(f"Example: {owned_games['genres_list'].iloc[0] if len(owned_games) > 0 else 'N/A'}")
elif not owned_games.empty and 'genres' in owned_games.columns:
    # Fallback for old CSV format
    owned_games['genres_list'] = owned_games['genres'].apply(parse_genres)
    print("✅ Parsed owned games genres (old format)")
    print(f"Example: {owned_games['genres_list'].iloc[0] if len(owned_games) > 0 else 'N/A'}")

# Parse owned games tags (NEW! - needed for building user profile)
if not owned_games.empty and 'tags' in owned_games.columns:
    owned_games['tags_dict'] = owned_games['tags'].apply(parse_tags)
    owned_games['top_tags'] = owned_games['tags_dict'].apply(
        lambda x: list(x.keys())[:30] if x else []  # Get top 30 tags per game
    )
    print("✅ Parsed owned games tags")
    print(f"Example tags: {owned_games['top_tags'].iloc[0][:5] if len(owned_games) > 0 and owned_games['top_tags'].iloc[0] else 'N/A'}")

# Parse catalog genres and tags
if not steam_catalog.empty:
    if 'genre' in steam_catalog.columns:
        steam_catalog['genres_list'] = steam_catalog['genre'].apply(parse_catalog_genre)
        print("\n✅ Parsed catalog genres")
    
    if 'tags' in steam_catalog.columns:
        steam_catalog['tags_dict'] = steam_catalog['tags'].apply(parse_tags)
        steam_catalog['top_tags'] = steam_catalog['tags_dict'].apply(
            lambda x: list(x.keys())[:5] if x else []
        )
        print("✅ Parsed catalog tags")
        print(f"Example tags: {steam_catalog['top_tags'].iloc[0] if len(steam_catalog) > 0 else 'N/A'}")

In [ ]:
if not owned_games.empty:
    print("=" * 60)
    print("YOUR GAMING LIBRARY ANALYSIS")
    print("=" * 60)
    print("\n📊 Dataset Info:")
    print(f"Total games owned: {len(owned_games)}")
    print(f"Total playtime: {owned_games['playtime_forever'].sum() / 60:.1f} hours")
    print(f"\nColumns available: {list(owned_games.columns)}")
    
    # Genre analysis
    if 'genres_list' in owned_games.columns:
        all_genres = []
        for genres in owned_games['genres_list']:
            all_genres.extend(genres)
        
        genre_counts = pd.Series(all_genres).value_counts()
        print(f"\n🎮 Your Top Genres (by game count):")
        print(genre_counts.head(10))
        
        # Genre weighted by playtime
        genre_playtime = {}
        for idx, row in owned_games.iterrows():
            playtime = row['playtime_forever'] / 60  # Convert to hours
            for genre in row['genres_list']:
                genre_playtime[genre] = genre_playtime.get(genre, 0) + playtime
        
        genre_playtime_series = pd.Series(genre_playtime).sort_values(ascending=False)
        print(f"\n⏱️  Your Top Genres (by playtime hours):")
        print(genre_playtime_series.head(10))

In [ ]:
# Visualize playtime and genre distribution
if not owned_games.empty and 'playtime_hours' in owned_games.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Playtime distribution
    axes[0, 0].hist(owned_games['playtime_hours'], bins=50, edgecolor='black', color='steelblue')
    axes[0, 0].set_xlabel('Playtime (hours)')
    axes[0, 0].set_ylabel('Number of Games')
    axes[0, 0].set_title('Distribution of Playtime')
    
    # Top 10 most played
    top_played = owned_games.nlargest(10, 'playtime_hours')
    axes[0, 1].barh(top_played['name'], top_played['playtime_hours'], color='coral')
    axes[0, 1].set_xlabel('Hours Played')
    axes[0, 1].set_title('Top 10 Most Played Games')
    axes[0, 1].invert_yaxis()
    
    # Genre distribution by game count
    if 'genres_list' in owned_games.columns:
        all_genres = []
        for genres in owned_games['genres_list']:
            all_genres.extend(genres)
        genre_counts = pd.Series(all_genres).value_counts().head(10)
        
        axes[1, 0].barh(genre_counts.index, genre_counts.values, color='lightgreen')
        axes[1, 0].set_xlabel('Number of Games')
        axes[1, 0].set_title('Your Top 10 Genres (by count)')
        axes[1, 0].invert_yaxis()
        
        # Genre distribution by playtime
        genre_playtime = {}
        for idx, row in owned_games.iterrows():
            playtime = row['playtime_forever'] / 60
            for genre in row['genres_list']:
                genre_playtime[genre] = genre_playtime.get(genre, 0) + playtime
        
        genre_playtime_series = pd.Series(genre_playtime).sort_values(ascending=False).head(10)
        axes[1, 1].barh(genre_playtime_series.index, genre_playtime_series.values, color='mediumpurple')
        axes[1, 1].set_xlabel('Total Hours Played')
        axes[1, 1].set_title('Your Top 10 Genres (by playtime)')
        axes[1, 1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()

## 3. Feature Engineering

Create features that indicate game preferences:
- Playtime categories (unplayed, tried, played, loved)
- Engagement score based on playtime

In [ ]:
if not owned_games.empty:
    # Create playtime categories
    def categorize_playtime(hours):
        if hours == 0:
            return 'unplayed'
        elif hours < 5:
            return 'tried'
        elif hours < 50:
            return 'played'
        else:
            return 'loved'
    
    owned_games['playtime_category'] = owned_games['playtime_hours'].apply(categorize_playtime)
    
    # Calculate engagement score (normalized playtime)
    max_playtime = owned_games['playtime_hours'].max()
    owned_games['engagement_score'] = owned_games['playtime_hours'] / max_playtime if max_playtime > 0 else 0
    
    print("📊 Playtime categories:")
    print(owned_games['playtime_category'].value_counts())
    
    # Extract preference profile from "loved" games (50+ hours)
    loved_games = owned_games[owned_games['playtime_category'] == 'loved']
    print(f"\n❤️  Your 'loved' games (50+ hours): {len(loved_games)}")
    
    if len(loved_games) > 0 and 'genres_list' in loved_games.columns:
        # Calculate genre preferences from loved games
        loved_genres = []
        for genres in loved_games['genres_list']:
            loved_genres.extend(genres)
        
        genre_preference = pd.Series(loved_genres).value_counts()
        print(f"\n🎯 Genres you LOVE (from 50+ hour games):")
        print(genre_preference.head(10))
        
        # Store for recommendation system
        owned_games['genre_preference_score'] = owned_games['genres_list'].apply(
            lambda genres: sum(genre_preference.get(g, 0) for g in genres)
        )

## 4. Simple Recommendation System

Start with a content-based approach using playtime as the main signal.
Later, this can be enhanced with game genres, tags, and features.

In [ ]:
def get_top_games(df, n=10):
    """Get user's most played games"""
    return df.nlargest(n, 'playtime_hours')[['name', 'playtime_hours', 'engagement_score']]

if not owned_games.empty:
    print("Your top games:")
    print(get_top_games(owned_games))

## 5. Build Recommendation System

Now we have all the data we need:
- **Your library**: Playtime + genres from Store API
- **Steam catalog**: 81k+ games with genres + tags from SteamSpy

**Recommendation Strategy**:
1. Analyze your "loved" games (50+ hours) to understand preferences
2. Extract genre patterns and tag preferences
3. Find games in catalog you don't own that match your taste
4. Rank by similarity to your loved games
5. Find games you do not like based on low playtime and the tags/genres and exclude similar games

## 5.1 Content-Based Filtering

finding out what games you dislike based on games you own with very low playtime and looking at their tags/genres to exclude similar games from recommendations. If tags/genres of disliked games overlap with tags/genres from liked games, we make sure to not count those overlaps negatively.

In [ ]:
if not owned_games.empty and 'playtime_category' in owned_games.columns:
    print("="*60)
    print("ANALYZING GAMES YOU DON'T PLAY")
    print("="*60)
    
    # Games you own but never played or barely tried (<5 hours)
    disliked_games = owned_games[owned_games['playtime_category'].isin(['unplayed', 'tried'])]
    
    print(f"\n📊 Games owned but barely/never played: {len(disliked_games)} out of {len(owned_games)}")
    print(f"    - Unplayed (0 hours): {len(owned_games[owned_games['playtime_category'] == 'unplayed'])}")
    print(f"    - Tried (<5 hours): {len(owned_games[owned_games['playtime_category'] == 'tried'])}")
    
    if len(disliked_games) > 0 and 'genres_list' in disliked_games.columns:
        # Extract genres from games you don't play
        disliked_genres = []
        for genres in disliked_games['genres_list']:
            disliked_genres.extend(genres)
        
        disliked_genre_counts = pd.Series(disliked_genres).value_counts()
        print(f"\n⚠️  Genres appearing most in games you DON'T play:")
        print(disliked_genre_counts.head(15))
        
        # Compare with genres you LOVE
        if len(loved_games) > 0:
            loved_genre_set = set()
            for genres in loved_games['genres_list']:
                loved_genre_set.update(genres)
            
            disliked_genre_set = set(disliked_genre_counts.index[:10])
            
            # Genres you don't play but appear often in unplayed games
            potential_exclusions = disliked_genre_set - loved_genre_set
            
            print(f"\n🚫 Suggested genres to EXCLUDE (appear in unplayed but not in loved games):")
            for genre in potential_exclusions:
                count = disliked_genre_counts.get(genre, 0)
                print(f"    - {genre} ({count} unplayed games)")
            
            # Check tags too
            if 'top_tags' in disliked_games.columns:
                disliked_tags = []
                for tags in disliked_games['top_tags']:
                    disliked_tags.extend(tags)
                
                disliked_tag_counts = pd.Series(disliked_tags).value_counts()
                print(f"\n⚠️  Top tags in games you DON'T play:")
                print(disliked_tag_counts.head(15))
    else:
        print("\n✅ Good news! You play most of your games. No clear dislikes to exclude.")
else:
    print("⚠️  Run the feature engineering cells first to categorize playtime")

## 5.2 Recommendation System

Build a user's profile and recommend games based on similarity to liked games while avoiding disliked game features.

In [ ]:
if not owned_games.empty and 'playtime_category' in owned_games.columns:
    loved_games = owned_games[owned_games['playtime_category'] == 'loved']
    
    if len(loved_games) > 0:
        print("="*60)
        print("YOUR LOVED GAMES - TAG ANALYSIS")
        print("="*60)
        
        # Show loved games and their tags
        print(f"\n❤️  Your {len(loved_games)} most-played games (50+ hours):\n")
        for idx, game in loved_games.nlargest(10, 'playtime_hours').iterrows():
            print(f"• {game['name']} ({game['playtime_hours']:.1f} hours)")
            if game.get('top_tags'):
                tags = game['top_tags'][:10]
                print(f"  Tags: {', '.join(tags)}")
            print()
        
        # Aggregate tags weighted by playtime
        tag_playtime = {}
        for idx, game in loved_games.iterrows():
            playtime = game['playtime_forever'] / 60
            game_tags = game.get('top_tags', [])
            for tag in game_tags:
                tag_playtime[tag] = tag_playtime.get(tag, 0) + playtime
        
        tag_playtime_series = pd.Series(tag_playtime).sort_values(ascending=False)
        
        print("="*60)
        print("TOP 30 TAGS (by playtime hours)")
        print("="*60)
        print(tag_playtime_series.head(30))
        
        # Identify noise tags
        nsfw_tags = {'Sexual Content', 'Nudity', 'NSFW', 'Adult', 'Hentai', 'Erotic'}
        meta_tags = {'Indie', 'Casual', 'Great Soundtrack', 'Singleplayer', 
                     'Multiplayer', 'Co-op', 'Classic'}
        
        gameplay_tags = {tag: hours for tag, hours in tag_playtime_series.items() 
                        if tag not in nsfw_tags and tag not in meta_tags}
        
        print(f"\n{'='*60}")
        print("TOP 20 GAMEPLAY TAGS (excluding NSFW and meta tags)")
        print("="*60)
        gameplay_series = pd.Series(gameplay_tags).sort_values(ascending=False)
        print(gameplay_series.head(20))
        
        print(f"\n💡 These tags define your gaming taste!")
        print(f"   The recommender will now match games with these specific tags.")
else:
    print("⚠️  Run the feature engineering cells first")

### NEW SCORING SYSTEM (Tag-Enhanced v2)

**How games are scored:**
1. **Tag Matching (45%)** - Specific features (Cyberpunk, Open World, FPS, Story Rich)
   - **MOST IMPORTANT** - Tags define the exact gameplay style you love
   - Filters out NSFW tags and meta tags (Indie, Casual, Singleplayer, etc.)
2. **Genre Matching (20%)** - Broad categories (Action, RPG, Strategy)
3. **Median Playtime (20%)** - How long players actually engage with the game
4. **Review Quality (15%)** - Community rating + volume (reduced to prevent popularity bias)

**Why tags dominate:**
- "Action RPG" could be Cyberpunk 2077 OR Diablo - completely different!
- Tags like "Cyberpunk", "Open World", "FPS", "Stealth" are specific
- Your profile learns from the EXACT features you played for 100+ hours

- Reviews are less important - we care what YOU like, not what's popular- Game development tools

- Software tools, animation/modeling tools

**Filters out non-games:**- Utilities (like Lossless Scaling)

### ⚠️ IMPORTANT: Filtering Out Non-Games

**Problem:** Tools like "Lossless Scaling" (Utilities genre) contaminate your profile
- These aren't games, but Steam counts playtime
- They add "Utilities" to your top genres, polluting recommendations

**Solution:** Automatically filtered out:
- Utilities, Software, Game Development tools
- Animation & Modeling, Audio Production
- Photo Editing, Video Production

**Your profile now ONLY learns from actual games!**

## 5.1 Analyze Your Loved Games' Tags

Let's see what specific tags appear in your most-played games to understand your preferences better

In [ ]:
# Build recommendation system using genres and tags
class GameRecommender:
    def __init__(self, user_games, catalog):
        self.user_games = user_games
        self.catalog = catalog
        self.user_profile = None
        self.owned_appids = set(user_games['appid'].astype(str))
        
        # NSFW content filter - tags to exclude by default
        self.nsfw_tags = {
            'Sexual Content', 'Nudity', 'NSFW', 'Adult',
            'Hentai', 'Erotic', 'Sexual', 'Porn', '18+', 'Adult Only'
        }
        
        # Non-gameplay genres (business model / development stage, not gameplay)
        # Can be optionally excluded from matching to focus on actual gameplay
        self.meta_genres = {
            'Indie', 'Casual', 'Early Access', 'Free to Play', 
            'Massively Multiplayer',  # Too broad - could be MMO RPG, MMO shooter, etc.
            'Utilities', 'Software', 'Animation & Modeling', 'Design & Illustration',  # NOT GAMES!
            'Audio Production', 'Video Production', 'Web Publishing', 'Education',
            'Photo Editing', 'Game Development'  # Tools/software, not games
        }
        
        # Meta tags that don't describe gameplay (similar to meta_genres)
        self.meta_tags = {
            'Indie', 'Casual', 'Free to Play', 'Early Access',
            'Great Soundtrack', 'Singleplayer', 'Multiplayer',  # Universal features, not specific
            'Co-op', 'Online Co-Op', 'PvP', 'PvE',  # Multiplayer modes, too generic
            'Moddable', 'Controller', 'Partial Controller Support',
            'Steam Achievements', 'Steam Cloud', 'Steam Trading Cards',
            'VR', 'VR Only',  # Hardware requirements, not gameplay style
            'Anime', 'Cute', 'Funny', 'Comedy',  # Aesthetic, not gameplay
            'Classic', 'Remake', 'Remaster', 'Retro'  # Historical context
        }
    
    def build_user_profile(self, include_meta_genres=False):
        """Analyze user's loved games to build preference profile
        
        Args:
            include_meta_genres: If True, include Indie/Casual/etc. in matching
                                 If False, focus only on gameplay genres (default)
        """
        loved_games = self.user_games[self.user_games['playtime_category'] == 'loved']
        
        if len(loved_games) == 0:
            print("⚠️  No 'loved' games found (50+ hours). Using all played games.")
            loved_games = self.user_games[self.user_games['playtime_hours'] > 0]
        
        # Genre preferences (weighted by playtime)
        genre_scores = {}
        for idx, game in loved_games.iterrows():
            playtime = game['playtime_forever'] / 60
            for genre in game['genres_list']:
                # Skip meta genres unless explicitly requested
                if not include_meta_genres and genre in self.meta_genres:
                    continue
                genre_scores[genre] = genre_scores.get(genre, 0) + playtime
        
        # TAG preferences (weighted by playtime) - NEW!
        tag_scores = {}
        for idx, game in loved_games.iterrows():
            playtime = game['playtime_forever'] / 60
            game_tags = game.get('top_tags', [])
            if not game_tags:
                continue
            
            for tag in game_tags:
                # Skip NSFW tags - they're noise
                if tag in self.nsfw_tags:
                    continue
                # Skip meta tags unless requested
                if not include_meta_genres and tag in self.meta_tags:
                    continue
                # Weight tags by playtime
                tag_scores[tag] = tag_scores.get(tag, 0) + playtime
        
        self.user_profile = {
            'loved_games': len(loved_games),
            'top_genres': pd.Series(genre_scores).sort_values(ascending=False).head(10).to_dict(),
            'top_tags': pd.Series(tag_scores).sort_values(ascending=False).head(30).to_dict(),  # Increased from 20 to capture more preferences
            'total_playtime': loved_games['playtime_forever'].sum() / 60,
            'include_meta_genres': include_meta_genres
        }
        
        print("🎯 User Profile Built!")
        print(f"Loved games: {self.user_profile['loved_games']}")
        genre_type = "all genres" if include_meta_genres else "gameplay genres only"
        print(f"Top genres ({genre_type}): {list(self.user_profile['top_genres'].keys())[:5]}")
        print(f"Top tags: {list(self.user_profile['top_tags'].keys())[:10]}")  # NEW!
        
        return self.user_profile
    
    def is_sfw_game(self, game):
        """Check if game is safe for work (no NSFW tags)"""
        game_tags = game.get('top_tags', [])
        if not game_tags:
            return True  # No tags means we can't filter it, allow by default
        
        # Check if any NSFW tags are present
        for tag in game_tags:
            if tag in self.nsfw_tags:
                return False
        return True
    
    def calculate_review_quality_score(self, positive, negative):
        """Calculate quality score based on review percentage and volume"""
        total_reviews = positive + negative
        if total_reviews == 0:
            return 0
        
        review_percentage = (positive / total_reviews) * 100
        
        # Quality tiers
        if review_percentage >= 95:
            quality_multiplier = 2.5  # Overwhelmingly Positive
        elif review_percentage >= 90:
            quality_multiplier = 2.0  # Very Positive
        elif review_percentage >= 80:
            quality_multiplier = 1.5  # Mostly Positive
        elif review_percentage >= 70:
            quality_multiplier = 1.0  # Positive
        elif review_percentage >= 60:
            quality_multiplier = 0.5  # Mixed (penalize)
        else:
            quality_multiplier = 0.1  # Negative/Mostly Negative (heavy penalty)
        
        # Volume bonus (logarithmic - more reviews = more reliable)
        volume_score = np.log10(total_reviews + 1) * quality_multiplier
        
        return volume_score
    
    def calculate_similarity(self, catalog_game):
        """Calculate similarity score between catalog game and user preferences"""
        if not self.user_profile:
            return 0
        
        score = 0
        include_meta = self.user_profile.get('include_meta_genres', False)
        
        # 1. Genre matching (20% weight)
        game_genres = catalog_game.get('genres_list', [])
        genre_match_score = 0
        for genre in game_genres:
            # Skip meta genres unless user wants them included
            if not include_meta and genre in self.meta_genres:
                continue
            # Match on genres in user's profile
            if genre in self.user_profile['top_genres']:
                genre_match_score += self.user_profile['top_genres'][genre] * 0.20
        score += genre_match_score
        
        # 2. TAG matching (45% weight) - MOST IMPORTANT! Defines exact gameplay style
        game_tags = catalog_game.get('top_tags', [])
        tag_match_score = 0
        if game_tags and 'top_tags' in self.user_profile:
            for tag in game_tags:
                # Skip NSFW and meta tags
                if tag in self.nsfw_tags:
                    continue
                if not include_meta and tag in self.meta_tags:
                    continue
                # Match on tags in user's profile
                if tag in self.user_profile['top_tags']:
                    tag_match_score += self.user_profile['top_tags'][tag] * 0.45
        score += tag_match_score
        
        # 3. Review quality score (15% weight - reduced to prevent popularity bias)
        positive = catalog_game.get('positive', 0)
        negative = catalog_game.get('negative', 0)
        quality_score = self.calculate_review_quality_score(positive, negative)
        score += quality_score * 15
        
        # 4. Engagement quality - median playtime (20% weight)
        median_playtime = catalog_game.get('median_forever', 0)
        if median_playtime > 0:
            median_hours = median_playtime / 60
            # Games with high median playtime = players actually enjoy them
            if median_hours >= 50:
                score += 50  # Deep, engaging game
            elif median_hours >= 20:
                score += 30  # Good engagement
            elif median_hours >= 10:
                score += 15  # Decent engagement
            elif median_hours >= 5:
                score += 5   # Some engagement
            # Games with <5 hour median might not be engaging enough
        
        return score
    
    # Change the min_reviews to determine how 'popular' the games you get recommended are
    
    def recommend(self, n=20, min_reviews=5000, min_review_score=70, sfw_only=True, 
                  exclude_early_access=True, include_meta_genres=False,
                  specific_genres=None, specific_tags=None,
                  exclude_genres=None, exclude_tags=None):
        """Generate top N recommendations
        
        Args:
            n: Number of recommendations to return
            min_reviews: Minimum positive reviews required
            min_review_score: Minimum review percentage (default 70% = "Positive" or better)
            sfw_only: If True, filter out NSFW content (default: True)
            exclude_early_access: If True, filter out Early Access games (default: True)
            include_meta_genres: If True, match on Indie/Casual/F2P/etc. (default: False)
            specific_genres: List of genres to filter by (e.g., ['RPG', 'Action'])
                            Only games with at least one of these genres will be recommended
            specific_tags: List of tags to filter by (e.g., ['Open World', 'Multiplayer'])
                          Only games with at least one of these tags will be recommended
            exclude_genres: List of genres to exclude (e.g., ['Sports', 'Racing'])
                           Games with ANY of these genres will be filtered out
            exclude_tags: List of tags to exclude (e.g., ['2D', 'Pixel Graphics'])
                         Games with ANY of these tags will be filtered out
        """
        if not self.user_profile or self.user_profile.get('include_meta_genres') != include_meta_genres:
            self.build_user_profile(include_meta_genres=include_meta_genres)
        
        print(f"\n🔍 Searching {len(self.catalog)} games for recommendations...")
        if sfw_only:
            print("🔒 SFW Filter: ON (excluding mature/sexual content)")
        if exclude_early_access:
            print("🚧 Excluding Early Access games (showing finished games only)")
        print(f"⭐ Minimum review score: {min_review_score}%")
        
        if include_meta_genres:
            print(f"🎮 Matching on ALL genres (including Indie/Casual/F2P/etc.)")
        else:
            print(f"🎮 Matching on gameplay genres only (ignoring Indie/Casual/etc.)")
        
        # Show specific filters if applied
        if specific_genres:
            print(f"🎯 Genre Filter: Only showing {', '.join(specific_genres)} games")
        if specific_tags:
            print(f"🏷️  Tag Filter: Only showing games with {', '.join(specific_tags)}")
        if exclude_genres:
            print(f"🚫 Excluding genres: {', '.join(exclude_genres)}")
        if exclude_tags:
            print(f"🚫 Excluding tags: {', '.join(exclude_tags)}")
        
        # Filter: games not owned, with minimum reviews, and genres available
        candidates = self.catalog[
            (~self.catalog['appid'].astype(str).isin(self.owned_appids)) &
            (self.catalog['positive'] >= min_reviews) &
            (self.catalog['genres_list'].apply(len) > 0)
        ].copy()
        
        # Apply specific genre filter (inclusion)
        if specific_genres:
            before_genre_filter = len(candidates)
            candidates = candidates[
                candidates['genres_list'].apply(
                    lambda genres: any(g in specific_genres for g in genres)
                )
            ]
            filtered = before_genre_filter - len(candidates)
            print(f"🎯 Filtered to {len(candidates)} games matching genre filter ({filtered} excluded)")
        
        # Apply specific tag filter (inclusion)
        if specific_tags:
            before_tag_filter = len(candidates)
            candidates = candidates[
                candidates['top_tags'].apply(
                    lambda tags: any(t in specific_tags for t in tags) if tags else False
                )
            ]
            filtered = before_tag_filter - len(candidates)
            print(f"🏷️  Filtered to {len(candidates)} games matching tag filter ({filtered} excluded)")
        
        # Apply genre exclusion filter (NEW!)
        if exclude_genres:
            before_exclude_filter = len(candidates)
            candidates = candidates[
                ~candidates['genres_list'].apply(
                    lambda genres: any(g in exclude_genres for g in genres)
                )
            ]
            filtered = before_exclude_filter - len(candidates)
            print(f"🚫 Filtered out {filtered} games with excluded genres")
        
        # Apply tag exclusion filter (NEW!)
        if exclude_tags:
            before_exclude_filter = len(candidates)
            candidates = candidates[
                ~candidates['top_tags'].apply(
                    lambda tags: any(t in exclude_tags for t in tags) if tags else False
                )
            ]
            filtered = before_exclude_filter - len(candidates)
            print(f"🚫 Filtered out {filtered} games with excluded tags")
        
        # Apply review quality filter
        candidates['review_percentage'] = (
            candidates['positive'] / (candidates['positive'] + candidates['negative']) * 100
        )
        before_review_filter = len(candidates)
        candidates = candidates[candidates['review_percentage'] >= min_review_score]
        print(f"⭐ Filtered out {before_review_filter - len(candidates)} low-rated games (<{min_review_score}%)")
        
        # Apply Early Access filter
        if exclude_early_access:
            if 'genres_list' in candidates.columns:
                before_ea_filter = len(candidates)
                candidates = candidates[
                    ~candidates['genres_list'].apply(lambda x: 'Early Access' in x)
                ]
                print(f"🚧 Filtered out {before_ea_filter - len(candidates)} Early Access games")
        
        # Apply SFW filter if enabled
        if sfw_only:
            before_filter = len(candidates)
            candidates = candidates[candidates.apply(self.is_sfw_game, axis=1)]
            filtered_count = before_filter - len(candidates)
            print(f"🔒 Filtered out {filtered_count} NSFW games")
        
        print(f"📋 {len(candidates)} candidate games after filtering")
        
        if len(candidates) == 0:
            print("❌ No games match your filters! Try relaxing some criteria.")
            return pd.DataFrame()
        
        # Calculate similarity scores
        candidates['similarity_score'] = candidates.apply(
            lambda row: self.calculate_similarity(row), axis=1
        )
        
        # Get top recommendations
        recommendations = candidates.nlargest(n, 'similarity_score')[
            ['appid', 'name', 'developer', 'publisher', 'genres_list', 'top_tags',
             'positive', 'negative', 'review_percentage', 'owners', 'median_forever', 
             'languages', 'price', 'initialprice', 'discount', 'similarity_score']
        ]
        
        return recommendations

# Initialize recommender
if not owned_games.empty and not steam_catalog.empty:
    print("="*60)
    print("BUILDING RECOMMENDATION SYSTEM")
    print("="*60)
    
    recommender = GameRecommender(owned_games, steam_catalog)
    
    # ========================================
    # EXAMPLE 1: General recommendations
    # ========================================
    print("\n" + "="*60)
    print("TOP 20 RECOMMENDED GAMES FOR YOU")
    print("="*60)
    recommendations = recommender.recommend(
        n=20, 
        sfw_only=True, 
        exclude_early_access=True,
        min_review_score=70,
        include_meta_genres=False  # Focus on gameplay genres only
    )
    
    # Display first 20 general recommendations
    for i, (idx, game) in enumerate(recommendations.head(20).iterrows(), 1):
        print(f"\n{'='*70}")
        print(f"#{i}. {game['name']}")
        print(f"{'='*70}")
        genres = game['genres_list'][:5] if len(game['genres_list']) > 5 else game['genres_list']
        print(f"🎮 Genres: {', '.join(genres)}")
        if game['top_tags'] and len(game['top_tags']) > 0:
            tags = game['top_tags'][:8]
            print(f"🏷️  Tags: {', '.join(tags)}")
        print(f"⭐ Reviews: {game['review_percentage']:.1f}% positive ({game['positive']:,} 👍)")
        print(f"🎯 Match Score: {game['similarity_score']:.1f}")
        print(f"🔗 Steam: https://store.steampowered.com/app/{game['appid']}/")

        #price info
        if game['price'] == 0:
            print("💰 Price: Free to Play")
        else:
            #if discounted
            #
            if pd.notna(game['discount']) and game['discount'] > 0:
                original_price_dollars = game['initialprice'] / 100  # Assuming price is in cents
                discounted_price_dollars = game['price'] / 100
                print(f"💰 Price: ${discounted_price_dollars:.2f} (Discounted from ${original_price_dollars:.2f})")

            price_dollars = game['price'] / 100  # Assuming price is in cents
            print(f"💰 Price: ${price_dollars:.2f}")
        
        #player metrics
        print(f"👥 Owners: {game['owners']}")
        print(f"⏱️  Median Playtime: {game['median_forever'] / 60:.1f} hours")

        #languages
        print(f"🗣️  Supported Languages: {game['languages']}")

        #review info
        print(f"⭐ Reviews: {game['review_percentage']:.1f}% positive ({game['positive']:,} 👍)")

        #dev and publisher
        print(f"🏢 Developer: {game['developer']}")
        print(f"🏢 Publisher: {game['publisher']}")
        
    
    # ========================================
    # EXAMPLE 2: Specific genre filter - RPG + Action only
    # ========================================
    print("\n\n" + "="*60)
    print("LOOKING FOR RPG & ACTION GAMES SPECIFICALLY")
    print("="*60)
    rpg_action_recs = recommender.recommend(
        n=10,
        sfw_only=True,
        exclude_early_access=True,
        min_review_score=70,
        include_meta_genres=False,  # Focus on gameplay genres only
        specific_genres=['RPG', 'Action']  # Only RPG or Action games
    )
    
    # Display first 20 RPG/Action recommendations
    for i, (idx, game) in enumerate(rpg_action_recs.head(20).iterrows(), 1):
        print(f"\n{'='*70}")
        print(f"#{i}. {game['name']}")
        print(f"{'='*70}")
        genres = game['genres_list'][:5] if len(game['genres_list']) > 5 else game['genres_list']
        print(f"🎮 Genres: {', '.join(genres)}")
        if game['top_tags'] and len(game['top_tags']) > 0:
            tags = game['top_tags'][:8]
            print(f"🏷️  Tags: {', '.join(tags)}")
        print(f"⭐ Reviews: {game['review_percentage']:.1f}% positive ({game['positive']:,} 👍)")
        print(f"🎯 Match Score: {game['similarity_score']:.1f}")
        
        # Price info
        if game['price'] == 0:
            print("💰 Price: Free to Play")
        else:
            if pd.notna(game['discount']) and game['discount'] > 0:
                original_price_dollars = game['initialprice'] / 100
                discounted_price_dollars = game['price'] / 100
                print(f"💰 Price: ${discounted_price_dollars:.2f} (was ${original_price_dollars:.2f}, {game['discount']}% off)")
            else:
                price_dollars = game['price'] / 100
                print(f"💰 Price: ${price_dollars:.2f}")
        
        # Player metrics & details
        print(f"👥 Owners: {game['owners']}")
        print(f"⏱️  Median Playtime: {game['median_forever'] / 60:.1f} hours")
        print(f"🏢 Developer: {game['developer']}")
        print(f"🔗 Steam: https://store.steampowered.com/app/{game['appid']}/")
    
    # ========================================
    # EXAMPLE 3: Specific tag filter - Open World games
    # ========================================
    print("\n\n" + "="*60)
    print("LOOKING FOR OPEN WORLD GAMES SPECIFICALLY")
    print("="*60)
    open_world_recs = recommender.recommend(
        n=10,
        sfw_only=True,
        exclude_early_access=True,
        min_review_score=80,  # Higher quality for open world
        include_meta_genres=False,  # Focus on gameplay genres only
        specific_tags=['Open World', 'Open World Survival Craft']
    )
    
    # Display first 20 Open World recommendations
    for i, (idx, game) in enumerate(open_world_recs.head(20).iterrows(), 1):
        print(f"\n{'='*70}")
        print(f"#{i}. {game['name']}")
        print(f"{'='*70}")
        genres = game['genres_list'][:5] if len(game['genres_list']) > 5 else game['genres_list']
        print(f"🎮 Genres: {', '.join(genres)}")
        if game['top_tags'] and len(game['top_tags']) > 0:
            tags = game['top_tags'][:8]
            print(f"🏷️  Tags: {', '.join(tags)}")
        print(f"⭐ Reviews: {game['review_percentage']:.1f}% positive ({game['positive']:,} 👍)")
        print(f"🎯 Match Score: {game['similarity_score']:.1f}")
        print(f"🔗 Steam: https://store.steampowered.com/app/{game['appid']}/")
    
    #price info
        if game['price'] == 0:
            print("💰 Price: Free to Play")
        else:
            #if discounted
            #
            if pd.notna(game['discount']) and game['discount'] > 0:
                original_price_dollars = game['initialprice'] / 100  # Assuming price is in cents
                discounted_price_dollars = game['price'] / 100
        
        # Price info
        if game['price'] == 0:
            print("💰 Price: Free to Play")
        else:
            if pd.notna(game['discount']) and game['discount'] > 0:
                original_price_dollars = game['initialprice'] / 100
                discounted_price_dollars = game['price'] / 100
                print(f"💰 Price: ${discounted_price_dollars:.2f} (was ${original_price_dollars:.2f}, {game['discount']}% off)")
            else:
                price_dollars = game['price'] / 100
                print(f"💰 Price: ${price_dollars:.2f}")
        
        # Player metrics & details
        print(f"👥 Owners: {game['owners']}")
        print(f"⏱️  Median Playtime: {game['median_forever'] / 60:.1f} hours")
        print(f"🏢 Developer: {game['developer']}")
        print(f"🔗 Steam: https://store.steampowered.com/app/{game['appid']}/")
    
    print("\n\n" + "="*60)
    print("💡 TIP: You can combine genre + tag filters!")
    print("Example: specific_genres=['RPG'], specific_tags=['Multiplayer']")
    print("="*60)
else:
    print("❌ Need both owned games and catalog data to generate recommendations")

## 6. Exclusion Based on Distinct Genre/Tag Patterns

In [ ]:
if not owned_games.empty and not steam_catalog.empty:
    print("\n" + "="*60)
    print("EXAMPLE: RECOMMENDATIONS EXCLUDING 2D GAMES")
    print("="*60)
    
    # Exclude games with '2D' tag, 'Pixel Graphics', and related tags
    no_2d_recs = recommender.recommend(
        n=20,
        sfw_only=True,
        exclude_early_access=True,
        min_review_score=70,
        include_meta_genres=False,
        exclude_tags=['2D', 'Pixel Graphics', 'Side Scroller', '2D Platformer']
    )
    
    # Display first 20 recommendations
    for i, (idx, game) in enumerate(no_2d_recs.head(20).iterrows(), 1):
        print(f"\n{'='*70}")
        print(f"#{i}. {game['name']}")
        print(f"{'='*70}")
        genres = game['genres_list'][:5] if len(game['genres_list']) > 5 else game['genres_list']
        print(f"🎮 Genres: {', '.join(genres)}")
        if game['top_tags'] and len(game['top_tags']) > 0:
            tags = game['top_tags'][:8]
            print(f"🏷️  Tags: {', '.join(tags)}")
        print(f"⭐ Reviews: {game['review_percentage']:.1f}% positive ({game['positive']:,} 👍)")
        print(f"🎯 Match Score: {game['similarity_score']:.1f}")
        
        # Price info
        if game['price'] == 0:
            print("💰 Price: Free to Play")
        else:
            if pd.notna(game['discount']) and game['discount'] > 0:
                original_price_dollars = game['initialprice'] / 100
                discounted_price_dollars = game['price'] / 100
                print(f"💰 Price: ${discounted_price_dollars:.2f} (was ${original_price_dollars:.2f}, {game['discount']}% off)")
            else:
                price_dollars = game['price'] / 100
                print(f"💰 Price: ${price_dollars:.2f}")
        
        print(f"👥 Owners: {game['owners']}")
        print(f"⏱️  Median Playtime: {game['median_forever'] / 60:.1f} hours")
        print(f"🔗 Steam: https://store.steampowered.com/app/{game['appid']}/")
    
    print("\n" + "="*60)
    print("💡 USAGE EXAMPLES:")
    print("="*60)
    print("\n# Exclude specific genres:")
    print("recommender.recommend(exclude_genres=['Sports', 'Racing'])")
    print("\n# Exclude specific tags:")
    print("recommender.recommend(exclude_tags=['2D', 'Pixel Graphics', 'Horror'])")
    print("\n# Combine exclusions with inclusions:")
    print("recommender.recommend(")
    print("    specific_genres=['RPG'],")
    print("    exclude_tags=['2D', 'Turn-Based']")
    print(")")
else:
    print("❌ Need both owned games and catalog data to generate recommendations")

In [ ]:
def diagnose_game(game_name, catalog, user_games, min_reviews=500, min_review_score=70):
    """Debug why a specific game isn't being recommended"""
    
    # Find the game in catalog
    game_matches = catalog[catalog['name'].str.contains(game_name, case=False, na=False)]
    
    if len(game_matches) == 0:
        print(f"❌ Game '{game_name}' not found in catalog")
        return
    
    if len(game_matches) > 1:
        print(f"⚠️  Multiple matches found for '{game_name}':")
        for idx, g in game_matches.iterrows():
            print(f"   - {g['name']} (appid: {g['appid']})")
        print("\nShowing analysis for first match...")
    
    game = game_matches.iloc[0]
    
    print(f"\n{'='*70}")
    print(f"DIAGNOSTIC: {game['name']}")
    print(f"{'='*70}")
    
    # Basic info
    print(f"\n📋 AppID: {game['appid']}")
    print(f"🎮 Genres: {', '.join(game['genres_list'])}")
    if game['top_tags']:
        print(f"🏷️  Tags: {', '.join(game['top_tags'][:8])}")
    
    # Check filters
    owned_appids = set(user_games['appid'].astype(str))
    is_owned = str(game['appid']) in owned_appids
    
    print(f"\n🔍 FILTER CHECKS:")
    print(f"   Already owned: {'❌ YES (filtered out)' if is_owned else '✅ NO'}")
    print(f"   Reviews: {game['positive']:,} positive ({game['positive'] + game['negative']:,} total)")
    print(f"   Meets min reviews ({min_reviews}): {'✅ YES' if game['positive'] >= min_reviews else f'❌ NO (only {game['positive']})'}")
    
    if game['positive'] + game['negative'] > 0:
        review_pct = (game['positive'] / (game['positive'] + game['negative'])) * 100
        print(f"   Review score: {review_pct:.1f}%")
        print(f"   Meets min score ({min_review_score}%): {'✅ YES' if review_pct >= min_review_score else f'❌ NO ({review_pct:.1f}%)'}")
    else:
        print(f"   Review score: N/A (no reviews)")
        print(f"   Meets min score: ❌ NO (no reviews)")
    
    is_early_access = 'Early Access' in game['genres_list']
    print(f"   Early Access: {'⚠️  YES (may be filtered)' if is_early_access else '✅ NO'}")
    
    # Calculate similarity score
    if hasattr(recommender, 'user_profile') and recommender.user_profile:
        score = recommender.calculate_similarity(game)
        print(f"\n🎯 SIMILARITY SCORE: {score:.1f}")
        
        # Show genre matches
        user_genres = recommender.user_profile['top_genres']
        matching_genres = [g for g in game['genres_list'] if g in user_genres and g not in recommender.meta_genres]
        if matching_genres:
            print(f"   Matching genres: {', '.join(matching_genres)}")
        else:
            print(f"   Matching genres: None (this is why score is low!)")
        
        print(f"\n   Your top genres: {list(user_genres.keys())[:5]}")
        print(f"   Game's genres: {game['genres_list']}")
    
    # Playtime metrics
    print(f"\n📊 ENGAGEMENT METRICS:")
    print(f"   Owners: {game['owners']}")
    print(f"   Median playtime: {game['median_forever'] / 60:.1f} hours")
    
    # Price
    if game['price'] == 0:
        print(f"   Price: Free to Play")
    else:
        print(f"   Price: ${game['price'] / 100:.2f}")
    
    print(f"\n🔗 Steam: https://store.steampowered.com/app/{game['appid']}/")
    print("="*70)

# Example usage - replace with games from your wishlist
if not owned_games.empty and not steam_catalog.empty:
    print("\n" + "="*60)
    print("WISHLIST DIAGNOSTIC TOOL")
    print("="*60)
    print("\nTo check why a game isn't recommended, use:")
    print("diagnose_game('Game Name', steam_catalog, owned_games)")
    print("\nExample:")
    print("diagnose_game('Elden Ring', steam_catalog, owned_games)")
    print("diagnose_game('Baldurs Gate 3', steam_catalog, owned_games)")
    print("\n" + "="*60)

diagnose_game('Clair Obscur: Expedition 33', steam_catalog, owned_games)

## 5.6 Diagnostic: Check Why Specific Games Aren't Recommended

Use this to debug why games you expect (from wishlist) aren't showing up

## 5.5 EXAMPLE: Using Exclusion Filters

Sometimes you know what you DON'T want. Let's exclude 2D games and see how it changes recommendations.